In [1]:
import pandas as pd 
import numpy as np
import datetime

In [ ]:
read_df = pd.read_csv("./csv/SPY.csv")
read_df.head()

In [ ]:
price_df = read_df.loc[:, ["Date", "Adj Close"]].copy()
price_df.head()

In [ ]:
#년월 컬럼을 생성
price_df["STD_YM"] = price_df["Date"].map(lambda x : \
    datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
price_df.head()

In [5]:
month_list = price_df["STD_YM"].unique()

In [ ]:
## 비어있는 데이터프레임 생성
month_last_df = pd.DataFrame()

for i in month_list:    ## 년월 리스트를 기준으로 반복
    # 기준 년월에 맞는 인덱스의 가장 마지막 날짜의 데이터를 month_last_df 추가
    month_last_df = month_last_df.append(price_df.loc\
        [price_df[price_df['STD_YM'] == i].index[-1], :])

month_last_df.set_index(['Date'], inplace=True)
month_last_df.head()

In [ ]:
## 컬럼 2개 생성
## 전월 종가 컬럼
## 전년도 종가 컬럼

month_last_df["BF_1M"] = month_last_df.shift(1)["Adj Close"]
month_last_df["BF_12M"] = month_last_df.shift(12)["Adj Close"]
month_last_df.fillna(0, inplace=True)
month_last_df.head(15)

In [ ]:
# 거래 내역 데이터프레임 생성
book = price_df.copy()
book.set_index(['Date'], inplace=True)
book["trade"] = ""
book.head()

In [ ]:
## 거래내역 추가

for i in month_last_df.index:
    signal = ""
    # 절대 모멘텀 계산
    momentum_index = month_last_df.loc[i, "BF_1M"] / month_last_df.loc[i, "BF_12M"] - 1
    # 절대 모멘텀 지표에 따라 True / False 판단
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf)\
         and (momentum_index != -np.inf)) else False
    if flag:
        signal = "buy"
    print("날짜 : ", i, '모멘텀 인덱스 : ', momentum_index, "flag : ", flag, "signal : ", signal)
    book.loc[i:, "trade"] = signal

In [ ]:
book.tail(20)

In [14]:
## 손익 계산
def returns(book):
    rtn = 1.0
    book["return"] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index:
        if book.loc[i, "trade"] == "buy" and book.shift(1).loc[i, "trade"] == "":   ## 구매한 시기
            buy = book.loc[i, "Adj Close"]
            print("진입일 : ", i , "진입 가격 : ", buy)
        elif book.loc[i, "trade"] == "" and book.shift(1).loc[i, "trade"] == "buy": ## 매도한 시기
            sell = book.loc[i, "Adj Close"]
            rtn = (sell - buy) / buy + 1 # 손익 계산
            book.loc[i, "return"] = rtn
            print("청산일 : ", i, "진입 가격 : ", buy, "청산 가격 : ", sell, " | return : ", round(rtn, 4))
        if book.loc[i, "trade"] == "":      ## buy, sell 초기화
            buy = 0.0
            sell = 0.0
    acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, "return"]
        acc_rtn = acc_rtn * rtn  ## 누적 수익률 계산
        book.loc[i, "acc_rtn"] = acc_rtn    ##누적 수익률 컬럼 생성
    print("누적 수익률 : ", round(acc_rtn, 4))
    return (round(acc_rtn, 4)) 

In [15]:
returns(book)

진입일 :  1994-01-31 진입 가격 :  30.045222999999996
청산일 :  1994-12-30 진입 가격 :  30.045222999999996 청산 가격 :  29.148033  | return :  0.9701
진입일 :  1995-02-28 진입 가격 :  31.357141
청산일 :  2000-12-29 진입 가격 :  31.357141 청산 가격 :  92.002388  | return :  2.934
진입일 :  2001-02-28 진입 가격 :  86.926689
청산일 :  2001-03-30 진입 가격 :  86.926689 청산 가격 :  82.055542  | return :  0.944
진입일 :  2003-07-31 진입 가격 :  72.26248199999999
청산일 :  2008-02-29 진입 가격 :  72.26248199999999 청산 가격 :  105.568001  | return :  1.4609
진입일 :  2009-10-30 진입 가격 :  85.141289
청산일 :  2011-10-31 진입 가격 :  85.141289 청산 가격 :  107.309486  | return :  1.2604
진입일 :  2011-11-30 진입 가격 :  106.873421
청산일 :  2012-01-31 진입 가격 :  106.873421 청산 가격 :  112.998047  | return :  1.0573
진입일 :  2012-02-29 진입 가격 :  117.902817
청산일 :  2015-10-30 진입 가격 :  117.902817 청산 가격 :  193.002396  | return :  1.637
진입일 :  2015-11-30 진입 가격 :  193.707825
청산일 :  2016-02-29 진입 가격 :  193.707825 청산 가격 :  180.733292  | return :  0.933
진입일 :  2016-04-29 진입 가격 :  193.650711
청산일 :  2016-05-31

10.9953